# Binary Classification using Convolutional Neural Networks

An investigation into the effects that image augmentation has on the accuracy and loss of Convolutional Neural Networks. *This work was completed as part of dissertation project for Bachelor of Science (Honours) in Computer Science with specialism in Artificial Intelligence.*

---

## Import Packages
Import all the necessary packages for the project to run.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os


## Control Variables
The below variables are used to control various elements of the network model. They have been concentrated in a single area to make experimental variation easier to achieve, they act as a "control panel" for the operation of the model.

`datasetName` - name of the experiment performed (used for graph titles etc.)

`resultsFileName` - prefix for all result filenames

`rotationRange` - range of rotations for a given experiment (e.g. 0°, 45°, 90°, 135°, 180°)

`categoryOne`, `categoryTwo` - name of first and second classification categories

`modelName` - name of the model used for graphs and results (combination of the dataset name and current rotation range setting

`datasetPath` - path to the image dataset master directory (containing sub-category directories)

`resultsPath` - path to results directory

`plotName` - title for all graph plots (by default using modelName)

`graphSize` - dimensions of the graph plots

`noEpochs` - number of epochs to run the model for

`initialLearningRate` - the learning rate used for searching for problem space  optima

`batchSize` - size of sample batches to be fed into the network model

`decayRate` - the decay rate of the learning rate (by default dR = lR / E)

`numberOfClases` - number of classification classes (binary classifier = 2)

`validationDatasetSize` - size of the dataset to be used for validation/testing of the accuracy of the model (usually 25%)

`randomSeed` - random number generation seed used for reproducibility

`imageHeight`, `imageWidth` - dimensions of the input images

`imageDepth` - Z dimension of the image (monochrome images = 1, colour = 3)

In [ ]:
resultsFileName = 'Demo'
datasetPath = 'Demo-dataset-rotation/'
rotationRange = 135
noEpochs = 100
initialLearningRate = 1e-5
batchSize = 32
decayRate = initialLearningRate / noEpochs
numberOfClasses = 2
validationDatasetSize = 0.25
randomSeed = 42
imageHeight = 64
imageWidth = 64
imageDepth = 3

resultsPath = 'Demo-results/'
modelName = resultsFileName + "-" + str(rotationRange)
plotName = modelName
graphSize = (15, 10)

In [ ]:
def file_is_image(path_to_file):
    filename, extension = os.path.splitext(path_to_file)
    if extension != '.jpg':
        return False
    else:
        return True


# Prints current timestamp, to be used in print statements
def stamp():
    time = "[" + str(datetime.datetime.now().time()) + "]   "
    return time


# Save final model performance
def save_network_stats(resultsPath, modelName, history, fileName, sensitivity, specificity, precision):
    # Extract data from history dictionary
    historyLoss = history.history['loss']
    historyLoss = str(historyLoss[-1])  # Get last value from loss
    historyAcc = history.history['acc']
    historyAcc = str(historyAcc[-1])  # Get last value from accuracy
    historyValLoss = history.history['val_loss']
    # Get last value from validated loss
    historyValLoss = str(historyValLoss[-1])
    historyValAcc = history.history['val_acc']
    # Get last value from validated accuracy
    historyValAcc = str(historyValAcc[-1])
    historyMSE = 0  # str(historyMSE[-1])
    historyMAPE = 0  # history.history['mape']
    historyMAPE = 0  # str(historyMAPE[-1])

    with open(resultsPath + fileName + ".txt", "a") as history_log:
        history_log.write(
            modelName + "," + historyLoss + "," + historyAcc + "," + historyValLoss + "," + historyValAcc + "," + str(
                noEpochs) + "," + str(initialLearningRate) + "," + str(historyMSE) + "," + str(
                historyMAPE) + "," + str(sensitivity) + "," + str(specificity) + "," + str(precision) + "\n")
    history_log.close()

    print(stamp() + "Keras Log Saved")

    print(history.history.keys())

    print(stamp() + "History File Saved")


# Build the network structure
def build_network_model(width, height, depth, classes):
    # Initialise the model
    model = Sequential()
    inputShape = (height, width, depth)

    # If 'channel first' is being used, update the input shape
    if K.image_data_format() == 'channel_first':
        inputShape = (depth, height, width)

    # First layer
    model.add(
        Conv2D(20, (5, 5), padding="same", input_shape=inputShape))  # Learning 20 (5 x 5) convolution filters
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Second layer
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Third layer - fully-connected layers
    model.add(Flatten())
    model.add(Dense(50))  # 500 nodes
    model.add(Activation("relu"))

    # Softmax classifier
    model.add(Dense(classes))  # number of nodes = number of classes
    model.add(Activation("softmax"))  # yields probability for each class

    # Return the model
    return model


# Calculate confusion matrix statistics
def calculate_statistics(tn, fp, fn, tp):
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    precision = tp / (tp + fp)

    return sensitivity, specificity, precision


# Save the confusion matrix as a graphical figure
def save_confusion_matrix(tp, tn, fp, fn):
    import seaborn as sns
    tp = int(tp)
    tn = int(tn)
    fp = int(fp)
    fn = int(fn)

    cm = [[tp, tn], [fp, fn]]
    cm = np.array(cm)
    heatmap = sns.heatmap(cm, annot=True, fmt='g', linewidths=0.2)
    fig = heatmap.get_figure()
    fig.savefig(resultsPath + '/' + modelName + '-confusion-matrix.png')


# Summarize history for accuracy
def save_accuracy_graph(history):
    plt.figure(figsize=graphSize, dpi=75)
    plt.grid(True, which='both')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.suptitle(modelName)
    plt.savefig(resultsPath + '/' + modelName + "-accuracy.png")
    plt.close()


# Summarize history for loss
def save_loss_graph(history):
    plt.figure(figsize=graphSize, dpi=75)
    plt.grid(True, which='both')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.suptitle(modelName)
    plt.savefig(resultsPath + '/' + modelName + "-loss.png")
    plt.close()


# Initialize the data and labels arrays
sortedData = []
sortedLabels = []
data = []
labels = []

# Go through dataset directory
print(stamp() + "Classifying the Dataset")
for datasetCategory in os.listdir(datasetPath):
    datasetCategoryPath = datasetPath + "/" + datasetCategory

    # Go through category 1 and then category 2 of the dataset
    for sample in os.listdir(datasetCategoryPath):
        # print(stamp() + sample)
        if file_is_image(datasetCategoryPath + "/" + sample):
            image = cv2.imread(datasetCategoryPath + "/" + sample)
            image = cv2.resize(image, (
                imageHeight, imageWidth))  # Network only accepts 28 x 28 so resize the image accordingly
            image = img_to_array(image)
            # Save image to the data list
            sortedData.append(image)

            # Decide on binary label
            if datasetCategory == 'benign':
                label = 1
            else:
                label = 0
            # Save label for the current image
            sortedLabels.append(label)

combined = list(zip(sortedData, sortedLabels))
random.shuffle(combined)
data[:], labels[:] = zip(*combined)

# Scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

validationDatasetLabels = []
# testSet = 0.25 * len(labels)
validationDatasetLabels = labels[-7:]

# Partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=7,  # Set manually to 7 for Demo
                                                  random_state=randomSeed)

# Convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=numberOfClasses)
testY = to_categorical(testY, num_classes=numberOfClasses)

# Construct the image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=rotationRange,
    fill_mode="nearest"
)

augValidation = ImageDataGenerator(
    rotation_range=rotationRange,
    fill_mode="nearest"
)

# Initialize the model
print(stamp() + "Compiling Network Model")

# Build the model based on control variable parameters
model = build_network_model(
    width=imageWidth, height=imageHeight, depth=imageDepth, classes=numberOfClasses)

# Set optimiser
opt = Adam(lr=initialLearningRate, decay=decayRate)

# Compile the model using binary crossentropy, preset optimiser and selected metrics
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy", "mean_squared_error", "mean_absolute_error"])
# Train the network
print(stamp() + "Training Network Model")

# Save results of training in history dictionary for statistical analysis
history = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=batchSize),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // batchSize,
    epochs=noEpochs,
    verbose=1)

# The following can be used to produce confusion matrices if necessary
# predictions = model.predict_classes(testX, batchSize, 0)
# tn, fp, fn, tp = confusion_matrix(validationDatasetLabels, predictions).ravel()
# print(tn, fp, fn, tp)
sensitivity, specificity, precision = 0, 0, 0   # Set to 0 if not used
# sensitivity, specificity, precision = calculate_statistics(tn, fp, fn, tp)

# Save all runtime statistics and plot graphs
save_network_stats(resultsPath, modelName, history,
                   resultsFileName, sensitivity, specificity, precision)
# save_confusion_matrix(tn, fp, fn, tp)
save_accuracy_graph(history)
save_loss_graph(history)

# Save the model to disk
print(stamp() + "Saving Network Model")
model_json = model.to_json()
with open(resultsPath + '/' + modelName + ".json", "w") as json_file:
    json_file.write(model_json)

# Save weights to disk
print(stamp() + "Saving Network Weights")
model.save_weights(resultsPath + '/' + modelName + ".h5", "w")


